In [ ]:
%%time

from __future__ import division

from pyomo.environ import *

import random as ra

for l in range(10):
    

    Model = ConcreteModel()

    costumers = 14

    depots = 5

    nodes = costumers + depots

    Model.n =  RangeSet(1,nodes)

    Model.n0 = set (i for i in Model.n if i <= depots)

    Model.nc = set (i for i in Model.n if i > depots )




    #def x1  (Model,i):
    #    return ra.randint(50,150)
    #Model.x1 = Param( Model.n , initialize = x1)

    x1 = {}
    for i in Model.n:
        x1[i] = ra.randint(50,100)
    Model.x1 = Param(Model.n,initialize = x1)


    #def y1 (Model,i):
    #    return ra.randint(50,150)
    #Model.y1 = Param(Model.n,initialize = y1)

    y1 = {}
    for i in Model.n:
         y1[i] = ra.randint(50,100)
    Model.y1 = Param(Model.n,initialize = y1)


    #def d (Model,i):
    #    return ra.randint(50,150)
    #Model.d = Param(Model.nc,initialize = d)

    d = {}
    for i in Model.nc :
    #    if i > depots :
            d[i] = ra.randint(50,100)
    Model.d = Param(Model.nc,initialize = d)

    #def p (Model,i):
    #    return ra.randint(50,150)
    #Model.p = Param(Model.nc,initialize = p)

    p = {}
    for i in Model.nc :
    #    if i > depots :
            p[i] = ra.randint(50,100)
    Model.p = Param(Model.nc,initialize = p)


    #def fd (Model,i):
    #    return ra.randint(50,100)
    #Model.p = Param(Model.n0,initialize = fd)

    fd ={}
    for i in Model.n0 :
    #    if i <depots+1:
            fd[i] = ra.randint(10,50)
    Model.fd = Param(Model.n0,initialize = fd)



    cd = 500

    cv = 200

    fv =20

    def c  (Model,i,j):
        return ((x1[i]-x1[j])**2+ (y1[i]-y1[j])**2)**0.5
    Model.c = Param( Model.n,Model.n , initialize = c)

    #c ={}
    #for i in Model.n:
    #    for j in Model.n:
    #                c[i,j] = ((x1[i]-x1[j])**2+ (y1[i]-y1[j])**2)**0.5
    #
    #Model.c = Param(Model.n,Model.n ,initialize = c)

    #Model.zz = Var(within = Reals)
    #Model.z3 =set( Model.nc , Model.n0)

    Model.x  = Var(Model.n , Model.n , within = Binary)

    Model.y = Var(Model.n0  , within = Binary)

    Model.z = Var(Model.nc , Model.n0  , within = Binary)
  

    Model.v = Var(Model.n,Model.n , within = PositiveReals)

    Model.u = Var(Model.n,Model.n , within = PositiveReals)

    #for i in Model.n:
    #    for k in Model.n:
    #        Model.z =

    def _2 (Model,i):
        xij=0
        for j in Model.n:
            xij+=Model.x[i,j]
            return xij ==1
    Model._2 = Constraint(Model.nc,rule =_2)

    #def _2 (Model):
    #
    #    for i in Model.nc:
    #        xij=0
    #       for j in Model.n:
    #           xij+=Model.x[i,j]
    #        return xij ==1
    #Model._2 = Constraint(rule =_2)

    def _3 (Model,i):
        xij = 0
        xji = 0
        for j in Model.n:
            xij+=Model.x[i,j]
            xji+=Model.x[j,i]
            return xij == xji
    Model._3 = Constraint(Model.n,rule =_3)


    #def _3 (Model):
    #    for i in Model.n:
    #        xij=0
    #        xji=0
    #        for j in Model.n:
    #            xij+=Model.x[i,j]
    #            xji+=Model.x[j,i]
    #        return xij ==xji
    #Model._3 = Constraint(rule =_3)

    def _4 (Model,i):

            uij = 0
            uji = 0
            for j in Model.n:
                uji +=Model.u[j,i]
                uij +=Model.u[i,j]
            return uji - uij == Model.d[i]
    Model._4 = Constraint(Model.nc,rule =_4)

    #def _4 (Model):
    #    for i in Model.nc:
    #        uij = 0
    #        uji = 0
    #        for j in Model.n:
    #            uji +=Model.u[j,i]
    #            uij +=Model.u[i,j]
    #        return uji - uij == Model.d[i]
    #Model._4 = Constraint(rule =_4)

    def _5 (Model,i):

            vij = 0
            vji = 0
            for j in Model.n:
                vji +=Model.v[j,i]
                vij +=Model.v[i,j]
            return vji - vij == Model.p[i]
    Model._5 = Constraint(Model.nc,rule =_5)

    #def _5 (Model):
    #    for i in Model.nc:
    #        vij = 0
    #        vji = 0
    #        for j in Model.n:
    #            vji +=Model.v[j,i]
    #            vij +=Model.v[i,j]
    #        return vji - vij == Model.p[i]
    #Model._5 = Constraint(rule =_5)

    def _6 (Model,i):
        for j in Model.n:
                if i!= j:
                    return  Model.u[i,j] + Model.v[i,j] <=cv * Model.x[i,j]
    Model._6 = Constraint(Model.n,rule =_6)

    #def _6 (Model):
    #    for i in Model.n:
    #        for j in Model.n:
    #            if i!= j:
    #                return  Model.u[i,j] + Model.v[i,j] <=cv * Model.x[i,j]
    #Model._6 = Constraint(rule =_6)

    def _7 (Model,k):

            ukj = 0
            zjkdj = 0
            for j in Model.nc:
                ukj += Model.u[k,j]
                zjkdj +=Model.z[j,k]*Model.d[j]
            return ukj == zjkdj
    Model._7 = Constraint(Model.n0,rule =_7)

    #def _7 (Model):
    #    for k in Model.nc:
    #        ukj = 0
    #        zjkdj = 0
    #        for j in Model.nc:
    #            ukj += Model.u[k,j]
    #            zjkdj +=Model.z[j,k]*Model.d[j]
    #        return ukj == zjkdj
    #Model._7 = Constraint(rule =_7)

    def _8 (Model,k):


            ujk=0
            for j in Model.nc:
                ujk+=Model.u[j,k]
            return ujk == 0
    Model._8 = Constraint(Model.n0,rule =_8)

    #def _8 (Model):
    #
    #    for k in Model.n0:
    #        ujk=0
    #        for j in Model.nc:
    #            ujk+=Model.u[j,k]
    #        return ujk == 0
    #Model._8 = Constraint(rule =_8)

    #def _9 (Model,k):
    #    vjk = 0
    #    zjkpj = 0
    #    for j in Model.nc:
    #        vjk += Model.u[k,j]
    #        zjkpj += Model.z[j,k]*Model.p[j]
    #    return vjk == zjkpj
    #Model._9 = Constraint(Model.n0,rule =_9)

    def _9 (Model):
        for k in Model.n0:
            vjk = 0
            zjkpj = 0
            for j in Model.nc:
                vjk += Model.u[k,j]
                zjkpj +=Model.z[j,k]*Model.p[j]
            return vjk == zjkpj
    Model._9 = Constraint(rule =_9)

    #def _10 (Model,k):
    #
    #        vjk=0
    #        for j in Model.nc:
    #            vjk+=Model.v[j,k]
    #        return vjk == 0
    #Model._10 = Constraint(Model.n0,rule =_10)

    def _10 (Model,k):


            vjk=0
            for j in Model.nc:
                vjk+=Model.v[j,k]
            return vjk == 0
    Model._10 = Constraint(Model.n0,rule =_10)

    #def _10 (Model):
    #
    #    for k in Model.n0:
    #        vjk=0
    #        for j in Model.nc:
    #            vjk+=Model.v[j,k]
    #        return vjk == 0
    #Model._10 = Constraint(rule =_10)

    def _11 (Model,i,j):

                return Model.u[i,j] <= (cv - Model.d[i])* Model.x[i,j]
    Model._11 = Constraint(Model.nc,Model.n,rule =_11)



    #def _11 (Model):
    #    for j in Model.n:
    #        for i in Model.nc:
    #            return Model.u[i,j] <= (cv - Model.d[i])* Model.x[i,j]
    #Model._11 = Constraint(rule =_11)

    def _12 (Model,i,j):

                return Model.v[i,j] <= (cv - Model.p[i])* Model.x[i,j]
    Model._12 = Constraint(Model.nc,Model.n,rule =_12)

    #def _12 (Model):
    #    for i in Model.nc:
    #        for j in Model.n:
    #            return Model.v[i,j] <= (cv - Model.p[i])* Model.x[i,j]
    #Model._12 = Constraint(rule =_12)

    #def _13 (Model,i):
    #    for j in Model.nc:
    #            return Model.u[i,j] >= Model.d[j]*Model.x[i,j]
    #Model._13 = Constraint(Model.n,rule =_13)

    def _13 (Model):
        for j in Model.nc:
            for i in Model.n:
                return Model.u[i,j] >= Model.d[j]*Model.x[i,j]
    Model._13 = Constraint(rule =_13)

    #def _14 (Model,i,j):
    #
    #            return Model.u[i,j] >= Model.d[j]*Model.x[i,j]
    #Model._14 = Constraint(Model.nc,Model.n,rule =_14)

    def _14 (Model):
        for j in Model.nc:
            for i in Model.n:
                return Model.v[i,j] >= Model.p[j] * Model.x[i,j]
    Model._14 = Constraint(rule =_14)

    def _15 (Model):

        for i in Model.nc:
            zik=0
            for k in Model.n0:
                zik += Model.z[i,k]
            return zik == 1
    Model._15 = Constraint(rule =_15)

    def _16 (Model):
        for k in Model.n0:
            dizik = 0
            for i in Model.nc:
                dizik += Model.d[i] * Model.z[i,k]
            return dizik <=cd * Model.y[k]
    Model._16 = Constraint(rule =_16)

    def _17 (Model):
        for k in Model.n0:
            pizik = 0
            for i in Model.nc:
                pizik += Model.d[i] * Model.z[i,k]
            return pizik <=cd * Model.y[k]
    Model._17 = Constraint(rule =_17)

    def _18 (Model):
        for k in Model.n0:
            for i in Model.nc:
                return Model.x[i,k] <= Model.z[i,k]
    Model._18 = Constraint(rule =_18)

    def _19 (Model):
        for k in Model.n0:
            for i in Model.nc:
                return Model.x[k,i] <= Model.z[i,k]
    Model._19 = Constraint(rule =_19)

    def _20 (Model):
        for k in Model.n0:
            for i in Model.nc:
                for j in Model.nc:
                    if i!=j :

                        zjm = 0
                        for m in Model.n0:
                            if m != k:
                                zjm += Model.z[j,m]
                        return Model.x[i,j]+Model.z[i,k] + zjm <=2
    Model._20 = Constraint(rule =_20)

    def obj (Model):
        cx = 0
        fdy = 0
        fvx = 0
        for i in Model.n :
            for j in Model.n :
                cx += Model.c[i,j]*Model.x[i,j]
        for k in Model.n0 :
            fdy += Model.fd[k] * Model.y[k]
        for k in Model.n0 :
            for i in Model.nc :
                fvx += fv * Model.x[k,i]
        return cx + fdy + fvx
    Model.objective = Objective(rule=obj)
    print('!'*60)

    def pyomo_postprocess(options=None, instance=None, results=None):
        Model.objective.display()
        

    #This is an optional code path that allows the script to be run outside of
    #pyomo command-line.  For example:  python transport.py
    if __name__ == '__main__':
       # This emulates what the pyomo command-line tools does
        from pyomo.opt import SolverFactory
    #   import pyomo.environ
        opt = SolverFactory("glpk")
        results = opt.solve(Model)
        #sends results to stdout
        results.write()
        print("\nDisplaying Solution\n" + '!'*60)
        pyomo_postprocess(None, Model, results)
    


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1086.33777571044
  Upper bound: 1086.33777571044
  Number of objectives: 1
  Number of constraints: 637
  Number of variables: 1115
  Number of nonzeros: 2537
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 241
      Number of created subproblems: 241
  Error rc: 0
  Time: 0.16698837280273438
# ----------------------------

In [1]:
%%time

from __future__ import division

from pyomo.environ import *

import random as ra

for l in range(10):
    

    Model = ConcreteModel()

    costumers = 14

    depots = 5

    nodes = costumers + depots

    Model.n =  RangeSet(1,nodes)

    Model.n0 = set (i for i in Model.n if i <= depots)

    Model.nc = set (i for i in Model.n if i > depots )




    #def x1  (Model,i):
    #    return ra.randint(50,150)
    #Model.x1 = Param( Model.n , initialize = x1)

    x1 = {}
    for i in Model.n:
        x1[i] = ra.randint(50,100)
    Model.x1 = Param(Model.n,initialize = x1)


    #def y1 (Model,i):
    #    return ra.randint(50,150)
    #Model.y1 = Param(Model.n,initialize = y1)

    y1 = {}
    for i in Model.n:
         y1[i] = ra.randint(50,100)
    Model.y1 = Param(Model.n,initialize = y1)


    #def d (Model,i):
    #    return ra.randint(50,150)
    #Model.d = Param(Model.nc,initialize = d)

    d = {}
    for i in Model.nc :
    #    if i > depots :
            d[i] = ra.randint(50,100)
    Model.d = Param(Model.nc,initialize = d)

    #def p (Model,i):
    #    return ra.randint(50,150)
    #Model.p = Param(Model.nc,initialize = p)

    p = {}
    for i in Model.nc :
    #    if i > depots :
            p[i] = ra.randint(50,100)
    Model.p = Param(Model.nc,initialize = p)


    #def fd (Model,i):
    #    return ra.randint(50,100)
    #Model.p = Param(Model.n0,initialize = fd)

    fd ={}
    for i in Model.n0 :
    #    if i <depots+1:
            fd[i] = ra.randint(10,50)
    Model.fd = Param(Model.n0,initialize = fd)



    cd = 500

    cv = 200

    fv =20

    def c  (Model,i,j):
        return ((x1[i]-x1[j])**2+ (y1[i]-y1[j])**2)**0.5
    Model.c = Param( Model.n,Model.n , initialize = c)

    #c ={}
    #for i in Model.n:
    #    for j in Model.n:
    #                c[i,j] = ((x1[i]-x1[j])**2+ (y1[i]-y1[j])**2)**0.5
    #
    #Model.c = Param(Model.n,Model.n ,initialize = c)

    #Model.zz = Var(within = Reals)
    #Model.z3 =set( Model.nc , Model.n0)

    Model.x  = Var(Model.n , Model.n , within = Binary)

    Model.y = Var(Model.n0  , within = Binary)

    Model.z = Var(Model.nc , Model.n0  , within = Binary)
  

    Model.v = Var(Model.n,Model.n , within = PositiveReals)

    Model.u = Var(Model.n,Model.n , within = PositiveReals)

    #for i in Model.n:
    #    for k in Model.n:
    #        Model.z =

    def _2 (Model,i):
        xij=0
        for j in Model.n:
            xij+=Model.x[i,j]
            return xij ==1
    Model._2 = Constraint(Model.nc,rule =_2)

    #def _2 (Model):
    #
    #    for i in Model.nc:
    #        xij=0
    #       for j in Model.n:
    #           xij+=Model.x[i,j]
    #        return xij ==1
    #Model._2 = Constraint(rule =_2)

    def _3 (Model,i):
        xij = 0
        xji = 0
        for j in Model.n:
            xij+=Model.x[i,j]
            xji+=Model.x[j,i]
            return xij == xji
    Model._3 = Constraint(Model.n,rule =_3)


    #def _3 (Model):
    #    for i in Model.n:
    #        xij=0
    #        xji=0
    #        for j in Model.n:
    #            xij+=Model.x[i,j]
    #            xji+=Model.x[j,i]
    #        return xij ==xji
    #Model._3 = Constraint(rule =_3)

    def _4 (Model,i):

            uij = 0
            uji = 0
            for j in Model.n:
                uji +=Model.u[j,i]
                uij +=Model.u[i,j]
            return uji - uij == Model.d[i]
    Model._4 = Constraint(Model.nc,rule =_4)

    #def _4 (Model):
    #    for i in Model.nc:
    #        uij = 0
    #        uji = 0
    #        for j in Model.n:
    #            uji +=Model.u[j,i]
    #            uij +=Model.u[i,j]
    #        return uji - uij == Model.d[i]
    #Model._4 = Constraint(rule =_4)

    def _5 (Model,i):

            vij = 0
            vji = 0
            for j in Model.n:
                vji +=Model.v[j,i]
                vij +=Model.v[i,j]
            return vji - vij == Model.p[i]
    Model._5 = Constraint(Model.nc,rule =_5)

    #def _5 (Model):
    #    for i in Model.nc:
    #        vij = 0
    #        vji = 0
    #        for j in Model.n:
    #            vji +=Model.v[j,i]
    #            vij +=Model.v[i,j]
    #        return vji - vij == Model.p[i]
    #Model._5 = Constraint(rule =_5)

    def _6 (Model,i):
        for j in Model.n:
                if i!= j:
                    return  Model.u[i,j] + Model.v[i,j] <=cv * Model.x[i,j]
    Model._6 = Constraint(Model.n,rule =_6)

    #def _6 (Model):
    #    for i in Model.n:
    #        for j in Model.n:
    #            if i!= j:
    #                return  Model.u[i,j] + Model.v[i,j] <=cv * Model.x[i,j]
    #Model._6 = Constraint(rule =_6)

    def _7 (Model,k):

            ukj = 0
            zjkdj = 0
            for j in Model.nc:
                ukj += Model.u[k,j]
                zjkdj +=Model.z[j,k]*Model.d[j]
            return ukj == zjkdj
    Model._7 = Constraint(Model.n0,rule =_7)

    #def _7 (Model):
    #    for k in Model.nc:
    #        ukj = 0
    #        zjkdj = 0
    #        for j in Model.nc:
    #            ukj += Model.u[k,j]
    #            zjkdj +=Model.z[j,k]*Model.d[j]
    #        return ukj == zjkdj
    #Model._7 = Constraint(rule =_7)

    def _8 (Model,k):


            ujk=0
            for j in Model.nc:
                ujk+=Model.u[j,k]
            return ujk == 0
    Model._8 = Constraint(Model.n0,rule =_8)

    #def _8 (Model):
    #
    #    for k in Model.n0:
    #        ujk=0
    #        for j in Model.nc:
    #            ujk+=Model.u[j,k]
    #        return ujk == 0
    #Model._8 = Constraint(rule =_8)

    #def _9 (Model,k):
    #    vjk = 0
    #    zjkpj = 0
    #    for j in Model.nc:
    #        vjk += Model.u[k,j]
    #        zjkpj += Model.z[j,k]*Model.p[j]
    #    return vjk == zjkpj
    #Model._9 = Constraint(Model.n0,rule =_9)

    def _9 (Model):
        for k in Model.n0:
            vjk = 0
            zjkpj = 0
            for j in Model.nc:
                vjk += Model.u[k,j]
                zjkpj +=Model.z[j,k]*Model.p[j]
            return vjk == zjkpj
    Model._9 = Constraint(rule =_9)

    #def _10 (Model,k):
    #
    #        vjk=0
    #        for j in Model.nc:
    #            vjk+=Model.v[j,k]
    #        return vjk == 0
    #Model._10 = Constraint(Model.n0,rule =_10)

    def _10 (Model,k):


            vjk=0
            for j in Model.nc:
                vjk+=Model.v[j,k]
            return vjk == 0
    Model._10 = Constraint(Model.n0,rule =_10)

    #def _10 (Model):
    #
    #    for k in Model.n0:
    #        vjk=0
    #        for j in Model.nc:
    #            vjk+=Model.v[j,k]
    #        return vjk == 0
    #Model._10 = Constraint(rule =_10)

    def _11 (Model,i,j):

                return Model.u[i,j] <= (cv - Model.d[i])* Model.x[i,j]
    Model._11 = Constraint(Model.nc,Model.n,rule =_11)



    #def _11 (Model):
    #    for j in Model.n:
    #        for i in Model.nc:
    #            return Model.u[i,j] <= (cv - Model.d[i])* Model.x[i,j]
    #Model._11 = Constraint(rule =_11)

    def _12 (Model,i,j):

                return Model.v[i,j] <= (cv - Model.p[i])* Model.x[i,j]
    Model._12 = Constraint(Model.nc,Model.n,rule =_12)

    #def _12 (Model):
    #    for i in Model.nc:
    #        for j in Model.n:
    #            return Model.v[i,j] <= (cv - Model.p[i])* Model.x[i,j]
    #Model._12 = Constraint(rule =_12)

    #def _13 (Model,i):
    #    for j in Model.nc:
    #            return Model.u[i,j] >= Model.d[j]*Model.x[i,j]
    #Model._13 = Constraint(Model.n,rule =_13)

    def _13 (Model):
        for j in Model.nc:
            for i in Model.n:
                return Model.u[i,j] >= Model.d[j]*Model.x[i,j]
    Model._13 = Constraint(rule =_13)

    #def _14 (Model,i,j):
    #
    #            return Model.u[i,j] >= Model.d[j]*Model.x[i,j]
    #Model._14 = Constraint(Model.nc,Model.n,rule =_14)

    def _14 (Model):
        for j in Model.nc:
            for i in Model.n:
                return Model.v[i,j] >= Model.p[j] * Model.x[i,j]
    Model._14 = Constraint(rule =_14)

    def _15 (Model):

        for i in Model.nc:
            zik=0
            for k in Model.n0:
                zik += Model.z[i,k]
            return zik == 1
    Model._15 = Constraint(rule =_15)

    def _16 (Model):
        for k in Model.n0:
            dizik = 0
            for i in Model.nc:
                dizik += Model.d[i] * Model.z[i,k]
            return dizik <=cd * Model.y[k]
    Model._16 = Constraint(rule =_16)

    def _17 (Model):
        for k in Model.n0:
            pizik = 0
            for i in Model.nc:
                pizik += Model.d[i] * Model.z[i,k]
            return pizik <=cd * Model.y[k]
    Model._17 = Constraint(rule =_17)

    def _18 (Model):
        for k in Model.n0:
            for i in Model.nc:
                return Model.x[i,k] <= Model.z[i,k]
    Model._18 = Constraint(rule =_18)

    def _19 (Model):
        for k in Model.n0:
            for i in Model.nc:
                return Model.x[k,i] <= Model.z[i,k]
    Model._19 = Constraint(rule =_19)

    def _20 (Model):
        for k in Model.n0:
            for i in Model.nc:
                for j in Model.nc:
                    if i!=j :

                        zjm = 0
                        for m in Model.n0:
                            if m != k:
                                zjm += Model.z[j,m]
                        return Model.x[i,j]+Model.z[i,k] + zjm <=2
    Model._20 = Constraint(rule =_20)

    def obj (Model):
        cx = 0
        fdy = 0
        fvx = 0
        for i in Model.n :
            for j in Model.n :
                cx += Model.c[i,j]*Model.x[i,j]
        for k in Model.n0 :
            fdy += Model.fd[k] * Model.y[k]
        for k in Model.n0 :
            for i in Model.nc :
                fvx += fv * Model.x[k,i]
        return cx + fdy + fvx
    Model.objective = Objective(rule=obj)
    print('!'*60)

    def pyomo_postprocess(options=None, instance=None, results=None):
        Model.objective.display()
        

    #This is an optional code path that allows the script to be run outside of
    #pyomo command-line.  For example:  python transport.py
    if __name__ == '__main__':
       # This emulates what the pyomo command-line tools does
        from pyomo.opt import SolverFactory
    #   import pyomo.environ
        opt = SolverFactory("glpk")
        results = opt.solve(Model)
        #sends results to stdout
        results.write()
        print("\nDisplaying Solution\n" + '!'*60)
        pyomo_postprocess(None, Model, results)
    


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1145.8746303088
  Upper bound: 1145.8746303088
  Number of objectives: 1
  Number of constraints: 637
  Number of variables: 1115
  Number of nonzeros: 2537
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 3727
      Number of created subproblems: 3727
  Error rc: 0
  Time: 2.37156343460083
# -------------------------------

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1328.47208647533
  Upper bound: 1328.47208647533
  Number of objectives: 1
  Number of constraints: 637
  Number of variables: 1115
  Number of nonzeros: 2537
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 9205
      Number of created subproblems: 9205
  Error rc: 0
  Time: 3.612004518508911
# ----------------------------------------------------------
#   Solution Information
# ---